In [ ]:
from ultralytics import YOLO
import cv2
from ultralytics.utils.plotting import Annotator
import os
import time
import torch, torch.nn as nn
from torch.nn.utils import prune

print('is cuda available:', torch.cuda.is_available())
print('cuda device count:', torch.cuda.device_count())
print('current device:', torch.cuda.current_device())
print('device name:', torch.cuda.get_device_name(0))

In [ ]:
version = 'v3'
# weight = os.path.join('models', version, 'pruned.pt')
weight = os.path.join('models', version, 'best.pt')
weight_abspath = os.path.abspath(weight)

print(weight_abspath)

model = YOLO(weight, task='detect')
# metrics = model.val(data=os.path.abspath('../merge_dataset/data.yaml'), plots=True, task='detect', conf=0.35)
# metrics = model.val(data=os.path.abspath('../old_dataset/data.yaml'), plots=True, task='detect', conf=0.35)
# model.export(format='onnx', int8=True, dynamic=True, simplify=True)
# model.export(format='openvino', int8=True)

In [ ]:
# print(metrics.box.map)    # map50-95 standard benchmark
# print(metrics.box.map50)  # map50
# print(metrics.box.map75)  # map75
# print(metrics.box.maps)

In [ ]:
# def sparsity(model):
#     # Return global model sparsity
#     a, b = 0, 0
#     for p in model.parameters():
#         a += p.numel()
#         b += (p == 0).sum()
#     return b / a

# pruning_param = 0.8

# for name, m in model.model.named_modules():
#     if isinstance(m, nn.Conv2d) or isinstance(m, torch.nn.Linear):
#         prune.l1_unstructured(m, name='weight', amount=pruning_param)  # prune
#         prune.remove(m, 'weight')  # make permanent
# print(f'Model pruned to {sparsity(model.model):.3g} global sparsity')

# ckpt = {
#     'model': model.model,
#     'train_args': {},  # save as dict
# }

# torch.save(ckpt, os.path.abspath(os.path.join('models', version, 'pruned.pt')))

In [ ]:
cap = cv2.VideoCapture(0)

# https://www.youtube.com/watch?v=S7dwu9xwKm4
# cap = cv2.VideoCapture('sample.mp4')

# Initialize FPS calculation
prev_time = 0

while cap.isOpened():
    ret, frame = cap.read()

    current_time = time.time()

    results = model.predict(frame, device='cuda:0', conf=0.35, stream=True, vid_stride=5, stream_buffer=True)
    # results = model.predict(frame, conf=0.35, stream=True, stream_buffer=True)

    for result in results:

        # https://stackoverflow.com/questions/75324341/yolov8-get-predicted-bounding-box
        annotator = Annotator(frame)

        boxes = result.boxes
        conf_arr = boxes.conf.cpu().numpy()
        conf = 0
        for box in boxes:
            b = box.xyxy[0]
            c = box.cls
            
            if len(conf_arr) > 0:
                conf = conf_arr[0]
            annotator.box_label(b, model.names[int(c)]+" "+f'{conf:.2f}')
    
    frame = annotator.result() 

    # Calculate FPS
    fps = 1 / (current_time - prev_time)
    prev_time = current_time

    # Display FPS on top-left corner of the window
    cv2.putText(frame, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Gun detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()